![Sơ đồ mô hình](mo_hinh_ck.jpg)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

## BƯỚC 1: Chia dữ liệu (Data Splitting)

In [3]:
# Load the dataset first
df = pd.read_csv("synthetic_model_training_data.csv")

# Step 1: Chia dữ liệu thành tập huấn luyện và kiểm tra

# Split dataset into training (80%) and testing (20%) with random_state=16
train_df, test_df = train_test_split(df, test_size=0.2, random_state=16)

# Display shape of datasets
train_df.shape, test_df.shape


((80, 11), (20, 11))

## Bước 2: Trích xuất đặc trưng từ K-NN (knn_feature)

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

# Chuẩn bị dữ liệu cho bước K-NN feature extraction
# Input: c1 (categorical), f1, f1x
X_knn = train_df[['c1', 'f1', 'f1x']].copy()

# Mã hóa biến phân loại c1 thành số
le_c1 = LabelEncoder()
X_knn['c1_encoded'] = le_c1.fit_transform(X_knn['c1'])
X_knn.drop('c1', axis=1, inplace=True)

# Đặt nhãn mục tiêu tạm thời để huấn luyện KNN - dùng c1 như mục tiêu để mô phỏng quá trình phân loại
y_knn = le_c1.transform(train_df['c1'])

# Huấn luyện mô hình KNN với 20 láng giềng
knn = KNeighborsClassifier(n_neighbors=20)
knn.fit(X_knn, y_knn)

# Tính xác suất dự đoán cho từng lớp (A, B, C)
knn_probs = knn.predict_proba(X_knn)

# Lấy xác suất lớn nhất cho mỗi hàng (maximum predicted probability)
knn_feature = knn_probs.max(axis=1)

# Lưu lại đặc trưng này vào tập huấn luyện
train_df['knn_feature'] = knn_feature

# Xem ví dụ vài dòng đầu
train_df[['knn_feature']].head()


,knn_feature
96,0.4
5,0.4
90,0.4
53,0.6
67,0.4


## Bước 3: Trích xuất đặc trưng từ Gaussian Naive Bayes (gnb_feature)

In [5]:
from sklearn.naive_bayes import GaussianNB

# Chuẩn bị dữ liệu cho bước GaussianNB feature extraction
# Input: c2 (categorical), f2, f2x
X_gnb = train_df[['c2', 'f2', 'f2x']].copy()

# Mã hóa c2 thành số
le_c2 = LabelEncoder()
X_gnb['c2_encoded'] = le_c2.fit_transform(X_gnb['c2'])
X_gnb.drop('c2', axis=1, inplace=True)

# Đặt nhãn mục tiêu là c2 đã mã hóa
y_gnb = le_c2.transform(train_df['c2'])

# Huấn luyện mô hình Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_gnb, y_gnb)

# Tính xác suất dự đoán cho từng lớp (X, Y)
gnb_probs = gnb.predict_proba(X_gnb)

# Lấy xác suất lớn nhất cho mỗi hàng
gnb_feature = gnb_probs.max(axis=1)

# Lưu lại đặc trưng này vào tập huấn luyện
train_df['gnb_feature'] = gnb_feature

# Xem ví dụ vài dòng đầu
train_df[['gnb_feature']].head()


,gnb_feature
96,1.0
5,1.0
90,1.0
53,1.0
67,1.0


## Bước 4: trích xuất đặc trưng từ f3, f4, f5, f6 và chuẩn hóa

In [6]:
# Tính trung bình
train_df['f_mean'] = train_df[['f3', 'f4', 'f5', 'f6']].mean(axis=1)

In [7]:
# Min-Max Normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train_df['f_mean'] = scaler.fit_transform(train_df[['f_mean']])

## Bước 5: Huấn luyện mô hình hồi quy tuyến tính (MLR)

In [8]:
from sklearn.linear_model import LinearRegression

# Tập đặc trưng đầu vào
X_train_final = train_df[['knn_feature', 'gnb_feature', 'f_mean']]

# Biến mục tiêu
y_train_final = train_df['limit']

# Khởi tạo và huấn luyện mô hình hồi quy tuyến tính
mlr = LinearRegression()
mlr.fit(X_train_final, y_train_final)

# Xem các hệ số mô hình
print("Hệ số hồi quy:", mlr.coef_)
print("Hệ số chệch (intercept):", mlr.intercept_)

Hệ số hồi quy: [-26.37434363   0.          -0.38717919]
Hệ số chệch (intercept): 41.59991090600797


## Bước 6: kiểm tra và đánh giá mô hình trên tập test


In [9]:
# Encode c1 and apply same transformations
X_test_knn = test_df[['c1', 'f1', 'f1x']].copy()
X_test_knn['c1_encoded'] = le_c1.transform(X_test_knn['c1'])
X_test_knn.drop('c1', axis=1, inplace=True)
knn_probs_test = knn.predict_proba(X_test_knn)
test_df['knn_feature'] = knn_probs_test.max(axis=1)


In [10]:
X_test_gnb = test_df[['c2', 'f2', 'f2x']].copy()
X_test_gnb['c2_encoded'] = le_c2.transform(X_test_gnb['c2'])
X_test_gnb.drop('c2', axis=1, inplace=True)
gnb_probs_test = gnb.predict_proba(X_test_gnb)
test_df['gnb_feature'] = gnb_probs_test.max(axis=1)


In [ ]:
test_df['f_mean'] = test_df[['f3', 'f4', 'f5', 'f6']].mean(axis=1)
test_df['f_mean'] = scaler.transform(test_df[['f_mean']])

In [12]:
from sklearn.metrics import r2_score, mean_absolute_error

# Dự đoán
X_test_final = test_df[['knn_feature', 'gnb_feature', 'f_mean']]
y_test_actual = test_df['limit']
y_test_pred = mlr.predict(X_test_final)

# Đánh giá
r2 = r2_score(y_test_actual, y_test_pred)
mae = mean_absolute_error(y_test_actual, y_test_pred)

print("R-squared:", r2)
print("MAE:", mae)


R-squared: 0.04012469803280749
MAE: 3.889630310736519
